In [0]:
# path to your train/test/meta folders
DATA_PATH = 'F:/Аудиоаналитика/'
    
# names of valuable files/folders
train_meta_fname = 'train.csv'
test_meta_fname = 'sample_submission.csv'
train_data_folder = 'audio'
test_data_folder = 'audio'

In [0]:
import numpy as np
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torchaudio
import torchvision
from torchaudio import transforms
from efficientnet_pytorch import EfficientNet
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from tqdm import tqdm

In [0]:
# set seeds
import random
import numpy as np

random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)
torch.backends.cudnn.deterministic = True

In [0]:
df_train = pd.read_csv(os.path.join(DATA_PATH, train_meta_fname))
df_test = pd.read_csv(os.path.join(DATA_PATH, test_meta_fname))
df_train.head(2)

,fname,label
0,8bcbcc394ba64fe85ed4.wav,Finger_snapping
1,00d77b917e241afa06f1.wav,Squeak


In [0]:
n_classes = df_train.label.nunique()
print(n_classes)
classes_dict = {cl:i for i,cl in enumerate(df_train.label.unique())}
df_train['label_encoded'] = df_train.label.map(classes_dict)
df_train.head()

41


,fname,label,label_encoded
0,8bcbcc394ba64fe85ed4.wav,Finger_snapping,0
1,00d77b917e241afa06f1.wav,Squeak,1
2,17bb93b73b8e79234cb3.wav,Electric_piano,2
3,7d5c7a40a936136da55e.wav,Harmonica,3
4,17e0ee7565a33d6c2326.wav,Snare_drum,4


In [0]:
class Model_1(nn.Module):
    
    def __init__(self, sample_rate=16000, n_classes=41):
        super().__init__()
        self.ms = torchaudio.transforms.MelSpectrogram(sample_rate)        
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=10, kernel_size=3, padding=1)
        self.cnn3 = nn.Conv2d(in_channels=10, out_channels=3, kernel_size=3, padding=1)        
        self.features = EfficientNet.from_pretrained('efficientnet-b0')            
        self.lin1 = nn.Linear(1000, 333)        
        self.lin2 = nn.Linear(333, 111)                
        self.lin3 = nn.Linear(111, n_classes)
        
    def forward(self, x):
        x = self.ms(x)                
        x = F.relu(self.cnn1(x))
        x = F.relu(self.cnn3(x))         
        x = self.features(x)
        x = x.view(x.shape[0], -1)
        x = F.relu(x)        
        x = F.relu(self.lin1(x))
        x = F.relu(self.lin2(x))
        x = self.lin3(x)
        return x
    
    def inference(self, x):
        x = self.forward(x)
        x = F.softmax(x)
        return x

In [0]:
class Model_2(nn.Module):
    
    def __init__(self, sample_rate=16000, n_classes=41):
        super().__init__()
        self.ms = torchaudio.transforms.MelSpectrogram(sample_rate, n_fft=1024, hop_length=256, n_mels=64)
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=10, kernel_size=3, padding=1)
        self.cnn3 = nn.Conv2d(in_channels=10, out_channels=3, kernel_size=3, padding=1)        
        self.features = EfficientNet.from_pretrained('efficientnet-b0')
        self.lin1 = nn.Linear(1000, 333)        
        self.lin2 = nn.Linear(333, 111)                
        self.lin3 = nn.Linear(111, n_classes)
        
    def forward(self, x):
        x = self.ms(x)                
        x = F.relu(self.cnn1(x))
        x = F.relu(self.cnn3(x))        
        x = self.features(x)
        x = x.view(x.shape[0], -1)
        x = F.relu(x)        
        x = F.relu(self.lin1(x))
        x = F.relu(self.lin2(x))
        x = self.lin3(x)   
        return x
    
    def inference(self, x):
        x = self.forward(x)
        x = F.softmax(x)
        return x

In [0]:
class Model_3(nn.Module):
    
    def __init__(self, sample_rate=16000, n_classes=41):
        super().__init__()
        self.ms = torchaudio.transforms.MFCC(sample_rate)
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=10, kernel_size=3, padding=1)
        self.cnn3 = nn.Conv2d(in_channels=10, out_channels=3, kernel_size=3, padding=1)        
        self.features = EfficientNet.from_pretrained('efficientnet-b0')            
        self.lin1 = nn.Linear(1000, 333)        
        self.lin2 = nn.Linear(333, 111)                
        self.lin3 = nn.Linear(111, n_classes)
        
    def forward(self, x):
        x = self.ms(x)
        x = F.relu(self.cnn1(x))
        x = F.relu(self.cnn3(x))         
        x = self.features(x)
        x = x.view(x.shape[0], -1)
        x = F.relu(x)        
        x = F.relu(self.lin1(x))
        x = F.relu(self.lin2(x))
        x = self.lin3(x)
        return x
    
    def inference(self, x):
        x = self.forward(x)
        x = F.softmax(x)
        return x

In [0]:
class Model_4(nn.Module):
    
    def __init__(self, sample_rate=16000, n_classes=41):
        super().__init__()
        self.ms = torchaudio.transforms.MelSpectrogram(sample_rate)        
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=10, kernel_size=3, padding=1)
        self.cnn3 = nn.Conv2d(in_channels=10, out_channels=15, kernel_size=3, padding=1)        
        self.cnn4 = nn.Conv2d(in_channels=15, out_channels=3, kernel_size=3, padding=1)        
        self.features = EfficientNet.from_pretrained('efficientnet-b0')            
        self.lin1 = nn.Linear(1000, 333)        
        self.lin2 = nn.Linear(333, 111)                
        self.lin3 = nn.Linear(111, n_classes)
        
    def forward(self, x):
        x = self.ms(x)
        x = F.relu(self.cnn1(x))
        x = F.relu(self.cnn3(x))        
        x = F.relu(self.cnn4(x))        
        x = self.features(x)
        x = x.view(x.shape[0], -1)
        x = F.relu(x)        
        x = F.relu(self.lin1(x))
        x = F.relu(self.lin2(x))
        x = self.lin3(x)  
        return x
    
    def inference(self, x):
        x = self.forward(x)
        x = F.softmax(x)
        return x

In [0]:
class Model_5(nn.Module):
    
    def __init__(self, sample_rate=16000, n_classes=41):
        super().__init__()
        self.ms = torchaudio.transforms.MFCC(sample_rate)
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=10, kernel_size=3, padding=1)
        self.cnn3 = nn.Conv2d(in_channels=10, out_channels=15, kernel_size=3, padding=1)        
        self.cnn4 = nn.Conv2d(in_channels=15, out_channels=3, kernel_size=3, padding=1)        
        self.features = EfficientNet.from_pretrained('efficientnet-b0')
        self.lin1 = nn.Linear(1000, 512)        
        self.lin2 = nn.Linear(512, 256)
        self.lin3 = nn.Linear(256, 128)                
        self.lin4 = nn.Linear(128, n_classes)
        
    def forward(self, x):
        x = self.ms(x)
        x = F.relu(self.cnn1(x))
        x = F.relu(self.cnn3(x))        
        x = F.relu(self.cnn4(x))        
        x = self.features(x)
        x = x.view(x.shape[0], -1)
        x = F.relu(x)  
        x = F.relu(self.lin1(x))
        x = F.relu(self.lin2(x))
        x = F.relu(self.lin3(x))
        x = self.lin4(x)
        return x
    
    def inference(self, x):
        x = self.forward(x)
        x = F.softmax(x)
        return x

In [0]:
def sample_or_pad(waveform, wav_len=32000):
    m, n = waveform.shape
    if n < wav_len:
        padded_wav = torch.zeros(1, wav_len)
        padded_wav[:, :n] = waveform
        return padded_wav
    elif n > wav_len:
        offset = np.random.randint(0, n - wav_len)
        sampled_wav = waveform[:, offset:offset+wav_len]
        return sampled_wav
    else:
        return waveform
        
class EventDetectionDataset(Dataset):
    def __init__(self, data_path, x, y=None):
        self.x = x
        self.y = y
        self.data_path = data_path
    
    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        path2wav = os.path.join(self.data_path, self.x[idx])
        waveform, sample_rate = torchaudio.load(path2wav, normalization=True)
        waveform = sample_or_pad(waveform)
        if self.y is not None:
            return waveform, self.y[idx]
        return waveform

In [0]:
X_train, X_val, y_train, y_val = train_test_split(df_train.fname.values, df_train.label_encoded.values, 
                                                  test_size=0.2, random_state=42)
train_loader = DataLoader(
                        EventDetectionDataset(os.path.join(DATA_PATH, train_data_folder), X_train, y_train),
                        batch_size=41
                )
val_loader = DataLoader(
                        EventDetectionDataset(os.path.join(DATA_PATH, train_data_folder), X_val, y_val),
                        batch_size=41
                )
test_loader = DataLoader(
                        EventDetectionDataset(os.path.join(DATA_PATH, test_data_folder), df_test.fname.values, None),
                        batch_size=41, shuffle=False
                )

In [0]:
def eval_model(model, eval_dataset):
    model.eval()
    forecast, true_labs = [], []
    with torch.no_grad():
        for wavs, labs in tqdm(eval_dataset):
            wavs, labs = wavs.cuda(), labs.detach().numpy()
            true_labs.append(labs)
            outputs = model.inference(wavs)
            
            outputs = outputs.detach().cpu().numpy().argmax(axis=1)
            forecast.append(outputs)
    forecast = [x for sublist in forecast for x in sublist]
    true_labs = [x for sublist in true_labs for x in sublist]
    return f1_score(forecast, true_labs, average='macro')

In [0]:
# Обучение моделей

criterion = nn.CrossEntropyLoss()
model = Model_5()
model = model.cuda()
lr = 1e-3

optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [0]:
n_epoch = 100
best_f1 = 0
for epoch in range(n_epoch):
    model.train()
    for wavs, labs in tqdm(train_loader):
        optimizer.zero_grad()
        wavs, labs = wavs.cuda(), labs.cuda()
        outputs = model(wavs)
        loss = criterion(outputs, labs)
        loss.backward()
        optimizer.step()
    f1 = eval_model(model, val_loader)
    f1_train = eval_model(model, train_loader)
    print(f'epoch: {epoch}, f1_test: {f1}, f1_train: {f1_train}')
    if f1 > best_f1:
        best_f1 = f1
        torch.save(model.state_dict(), '../baseline_v5.pt')
        
    lr = lr * 0.95
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [0]:
# make a model
model_name = 'baseline_v5.pt'
model = BaseLineModel().cuda()
model.load_state_dict(torch.load(os.path.join('..', model_name)))
model.eval()
forecast = []
with torch.no_grad():
    for wavs in tqdm(test_loader):
        wavs = wavs.cuda()
        outputs = model.inference(wavs)
        outputs = outputs.detach().cpu().numpy().argmax(axis=1)
        forecast.append(outputs)
forecast = [x for sublist in forecast for x in sublist]
decoder = {classes_dict[cl]:cl for cl in classes_dict}
forecast = pd.Series(forecast).map(decoder)
df_test['label'] = forecast
df_test.to_csv(f'{model_name}.csv', index=None)

In [0]:
def model_predict(model, data_loader):
    forecast = []
    probs = []
    with torch.no_grad():
        for wavs, labs in tqdm(data_loader):
            wavs = wavs.cuda()
            outputs = model.inference(wavs)
            probs.append(outputs.detach().cpu().numpy())
            outputs = outputs.detach().cpu().numpy().argmax(axis=1)
            forecast.append(outputs)
    forecast = [x for sublist in forecast for x in sublist]
    probs = [x for sublist in probs for x in sublist]
    decoder = {classes_dict[cl]:cl for cl in classes_dict}
    forecast = pd.Series(forecast).map(decoder)
    
    return forecast, probs

In [0]:
def model_predict_test(model, test_loader):
    forecast = []
    probs = []
    with torch.no_grad():
        for wavs in tqdm(test_loader):
            wavs = wavs.cuda()
            outputs = model.inference(wavs)
            probs.append(outputs.detach().cpu().numpy())
            outputs = outputs.detach().cpu().numpy().argmax(axis=1)
            forecast.append(outputs)
    forecast = [x for sublist in forecast for x in sublist]
    probs = [x for sublist in probs for x in sublist]
    decoder = {classes_dict[cl]:cl for cl in classes_dict}
    forecast = pd.Series(forecast).map(decoder)
    
    return forecast, probs

In [0]:
def model_train(model, train_loader, val_loader, name, learning_rate, epoch):
    n_epoch = epoch
    best_f1 = 0
    for epoch in range(n_epoch):
        model.train()
        for wavs, labs in tqdm(train_loader):
            optimizer.zero_grad()
            wavs, labs = wavs.cuda(), labs.cuda()
            outputs = model(wavs)
            loss = criterion(outputs, labs)
            loss.backward()
            optimizer.step()
        f1 = eval_model(model, val_loader)
        f1_train = eval_model(model, train_loader)
        print(f'epoch: {epoch}, f1_test: {f1}, f1_train: {f1_train}')
        if f1 > best_f1:
            best_f1 = f1
            torch.save(model.state_dict(), '../' + name)
        
        learning_rate = learning_rate * 0.95
        for param_group in optimizer.param_groups:
            param_group['lr'] = learning_rate

# Semi-supervised learning

In [0]:
model_name = 'baseline_v4.pt'
model = Model_4().cuda()
model.load_state_dict(torch.load(os.path.join('..', model_name)))
model.eval()
_, probs = model_predict_test(model, test_loader)

In [0]:
probmax = [max(x) for x in probs]

idx_max = []
idx_else = []
for i, prob in enumerate(probmax):
    if (prob) > 0.99:
        idx_max.append(i)
    else:
        idx_else.append(i)

In [0]:
# Создание нового train

df_train_new = pd.read_csv(os.path.join(DATA_PATH, train_meta_fname))
df_train_new['label_encoded'] = df_train_new.label.map(classes_dict)

for i in idx_max:
    df_train_new.loc[len(df_train_new)] = [df_test['fname'][i], df_test['label'][i], classes_dict[df_test['label'][i]]]
    
df_train_new.to_csv('train_with_test_99_v4.csv', index=None) 

In [0]:
# Создание нового test

df_test_new = pd.DataFrame(columns = ['fname', 'label'])

for i in idx_else:
    df_test_new.loc[len(df_test_new)] = [df_test['fname'][i], 0.0]
    
df_test_new.to_csv('test_new_99_v4.csv', index=None)

In [0]:
X_train, X_val, y_train, y_val = train_test_split(df_train_new.fname.values, df_train_new.label_encoded.values, 
                                                  test_size=0.2, random_state=42)
train_loader_new = DataLoader(
                        EventDetectionDataset(os.path.join(DATA_PATH, train_data_folder), X_train, y_train),
                        batch_size=41
                )
val_loader_new = DataLoader(
                        EventDetectionDataset(os.path.join(DATA_PATH, train_data_folder), X_val, y_val),
                        batch_size=41
                )
test_loader_new = DataLoader(
                        EventDetectionDataset(os.path.join(DATA_PATH, test_data_folder), df_test_new.fname.values, None),
                        batch_size=41, shuffle=False
                )

In [0]:
criterion = nn.CrossEntropyLoss()
model = Model_4()
model = model.cuda()
model_name = 'Semi_S_99_v4.pt'
lr = 1e-3

optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [0]:
model_train(model, train_loader_new, val_loader_new, model_name, lr, 100)

In [0]:
model_name = 'Semi_S_99_v4.pt'
model = Model_4().cuda()
model.load_state_dict(torch.load(os.path.join('..', model_name)))
model.eval()
forecast, _ = model_predict_test(model, test_loader)

df_test['label'] = forecast
df_test.to_csv(f'{model_name}.csv', index=None)

In [0]:
_, probs = model_predict_test(model, test_loader_new)

In [0]:
probmax = [max(x) for x in probs]

idx_max = []
idx_else = []
for i, prob in enumerate(probmax):
    if (prob) > 0.99:
        idx_max.append(i)
    else:
        idx_else.append(i)

In [0]:
df_train_new_2 = pd.read_csv(os.path.join('', 'train_with_test_99_v4.csv'))
df_train_new_2['label_encoded'] = df_train_new_2.label.map(classes_dict)

for i in idx_max:
    df_train_new_2.loc[len(df_train_new_2)] = [df_test_new['fname'][i], df_test_new['label'][i], classes_dict[df_test_new['label'][i]]]
    
df_train_new_2.to_csv('train_with_test_99_v4_2.csv', index=None) 

In [0]:
df_test_new_2 =pd.DataFrame(columns = ['fname', 'label'])

for i in idx_else:
    df_test_new_2.loc[len(df_test_new_2)] = [df_test_new['fname'][i], 0.0]
    
df_test_new_2.to_csv('test_new_99_v4_2.csv', index=None)

In [0]:
X_train, X_val, y_train, y_val = train_test_split(df_train_new_2.fname.values, df_train_new_2.label_encoded.values, 
                                                  test_size=0.2, random_state=42)
train_loader_new_2 = DataLoader(
                        EventDetectionDataset(os.path.join(DATA_PATH, train_data_folder), X_train, y_train),
                        batch_size=41
                )
val_loader_new_2 = DataLoader(
                        EventDetectionDataset(os.path.join(DATA_PATH, train_data_folder), X_val, y_val),
                        batch_size=41
                )
test_loader_new_2 = DataLoader(
                        EventDetectionDataset(os.path.join(DATA_PATH, test_data_folder), df_test_new_2.fname.values, None),
                        batch_size=41, shuffle=False
                )

In [0]:
# Продолжение обучения на новых данных

model_name = 'Semi_S_99_v4_2step_continie_learning.pt'
model_train(model, train_loader_new_2, val_loader_new_2, model_name, lr, 50)

In [0]:
model_name = 'Semi_S_99_v4_2step_continie_learning.pt'
model = Model_4().cuda()
model.load_state_dict(torch.load(os.path.join('..', model_name)))
model.eval()
forecast, _ = model_predict_test(model, test_loader)

df_test['label'] = forecast
df_test.to_csv(f'{model_name}.csv', index=None)

# EM

In [0]:
# Загрузить модель

model_name = 'baseline_v4.pt'
model = Model_4().cuda()
model.load_state_dict(torch.load(os.path.join('..', model_name)))
model.eval()
forecast, _ = model_predict_test(model, test_loader)
df_test['label'] = forecast

In [0]:
# добавление test к train

df_train_new = pd.read_csv(os.path.join(DATA_PATH, train_meta_fname))
df_train_new['label_encoded'] = df_train_new.label.map(classes_dict)

for i in range(len(df_test)):
    df_train_new.loc[len(df_train_new)] = [df_test['fname'][i], df_test['label'][i], classes_dict[df_test['label'][i]]]
    
df_train_new.to_csv('train_with_test_full_v4.csv', index=None) 

In [0]:
df_test_new = pd.read_csv(os.path.join(DATA_PATH, test_meta_fname))

In [0]:
X_train, X_val, y_train, y_val = train_test_split(df_train_new.fname.values, df_train_new.label_encoded.values, 
                                                  test_size=0.2, random_state=42)
train_loader_new = DataLoader(
                        EventDetectionDataset(os.path.join(DATA_PATH, train_data_folder), X_train, y_train),
                        batch_size=41
                )
val_loader_new = DataLoader(
                        EventDetectionDataset(os.path.join(DATA_PATH, train_data_folder), X_val, y_val),
                        batch_size=41
                )

In [0]:
# Продолжение обучения

model_name = 'EM_v4_continie_learning.pt'
model_train(model, train_loader_new, val_loader_new, model_name, lr, 30)

In [0]:
model_name = 'EM_v4_continie_learning.pt'
model = Model_4().cuda()
model.load_state_dict(torch.load(os.path.join('..', model_name)))
model.eval()
forecast, _ = model_predict_test(model, test_loader)

df_test_new['label'] = forecast
df_test_new.to_csv(f'{model_name}.csv', index=None)

In [0]:
# сравнение старого test и test_new

csv = ['baseline_v4.pt.csv', 'EM_v4_continie_learning.pt.csv']
df = pd.DataFrame()

for i in range(len(csv)):
    temp_df = pd.read_csv(os.path.join('', csv[i]))
    df['label' + str(i)] = temp_df.label   

print('Различных значений:', sum(df['label0'] != df['label1']))

Различных значений: 764


In [0]:
# обновление train

df_train_new_2 = pd.read_csv(os.path.join(DATA_PATH, train_meta_fname))
df_train_new_2['label_encoded'] = df_train_new_2.label.map(classes_dict)

for i in range(len(df_test_new)):
    df_train_new_2.loc[len(df_train_new_2)] = [df_test_new['fname'][i], df_test_new['label'][i], classes_dict[df_test_new['label'][i]]]
    
df_train_new_2.to_csv('train_with_test_full_v4_step2.csv', index=None) 

In [0]:
df_test_new_2 = pd.read_csv(os.path.join(DATA_PATH, test_meta_fname))

In [0]:
X_train, X_val, y_train, y_val = train_test_split(df_train_new_2.fname.values, df_train_new_2.label_encoded.values, 
                                                  test_size=0.2, random_state=42)
train_loader_new_2 = DataLoader(
                        EventDetectionDataset(os.path.join(DATA_PATH, train_data_folder), X_train, y_train),
                        batch_size=41
                )
val_loader_new_2 = DataLoader(
                        EventDetectionDataset(os.path.join(DATA_PATH, train_data_folder), X_val, y_val),
                        batch_size=41
                )

In [0]:
# Продолжение обучения

model_name = 'EM_v4_step2_continie_learning.pt'
model_train(model, train_loader_new_2, val_loader_new_2, model_name, lr, 20)

In [0]:
model_name = 'EM_v4_step2_continie_learning.pt'
model = Model_4().cuda()
model.load_state_dict(torch.load(os.path.join('..', model_name)))
model.eval()
forecast, _ = model_predict_test(model, test_loader)

df_test_new_2['label'] = forecast
df_test_new_2.to_csv(f'{model_name}.csv', index=None)

In [0]:
df = pd.DataFrame()

df['label' + str(0)] = df_test.label
df['label' + str(1)] = df_test_new.label
df['label' + str(2)] = df_test_new_2.label 

In [0]:
print('Различных значений:', sum(df['label1'] != df['label2']))

Различных значений: 705


# Голосование

In [0]:
def voiting(csv, name):    
    df = pd.DataFrame()
    
    for i in range(len(csv)):
        temp_df = pd.read_csv(os.path.join('', csv[i]))
        df['label' + str(i)] = temp_df.label   
    
    df_test['label'] = df.mode(axis='columns')[0]
    df_test.to_csv(name, index=None)
    
    return df

In [0]:
csv_arr = ['baseline_v5.pt.csv', 'baseline_v3.pt.csv', 'baseline_v2.pt.csv']
labels_result = voiting(csv_arr, 'voting_v5_3_2.csv')

In [0]:
csv_arr = ['Stacking_v1_2_3_4_5_SS41_EM42.csv', 'Stacking_v1_2_3_4_5_SS41_SS42_EM41_EM42.csv', 'Stacking_v1_4_SS41_EM42.csv', 'Stacking_v2_3_4_5.csv', 'Stacking_vSS41_SS42_EM41_EM42.csv']
labels_result = voiting(csv_arr, 'Voting_v_Stacking_all.csv')

In [0]:
# Варианты csv

# 'baseline_v4.pt.csv', 'baseline_fulldiv.pt.csv', 'baseline_v5.pt.csv', 'baseline_v3.pt.csv', 'baseline_v2.pt.csv'
# 'Semi_S_99_v4.pt.csv','Semi_S_99_v4_2step_continie_learning.pt.csv','EM_v4_continie_learning.pt.csv','EM_v4_step2_continie_learning.pt.csv'
# 'Stacking_v1_2_3_4_5_SS41_EM42.csv', 'Stacking_v1_2_3_4_5_SS41_SS42_EM41_EM42.csv', 'Stacking_v1_4_SS41_EM42.csv', 'Stacking_v2_3_4_5.csv', 'Stacking_vSS41_SS42_EM41_EM42.csv'

# Stacking

In [0]:
# Предсказания моделей на train, val и test
# Разные data_loader и функции

In [0]:
model_name = 'Semi_S_99_v4_2step_continie_learning.pt'
model = Model_4().cuda()
model.load_state_dict(torch.load(os.path.join('..', model_name)))
model.eval()
_, probs_1 = model_predict_test(model, test_loader)

In [0]:
model_name = 'EM_v4_continie_learning.pt'
model = Model_4().cuda()
model.load_state_dict(torch.load(os.path.join('..', model_name)))
model.eval()
_, probs_2 = model_predict_test(model, test_loader)

In [0]:
model_name = 'Semi_S_99_v4.pt'
model = Model_4().cuda()
model.load_state_dict(torch.load(os.path.join('..', model_name)))
model.eval()
_, probs_3 = model_predict_test(model, test_loader)

In [0]:
model_name = 'EM_v4_step2_continie_learning.pt'
model = Model_4().cuda()
model.load_state_dict(torch.load(os.path.join('..', model_name)))
model.eval()
_, probs_4 = model_predict_test(model, test_loader)

In [0]:
# Объединение предсказаний

In [0]:
probs_train = np.hstack((probs_1, probs_2, probs_3, probs_4))

In [0]:
probs_val = np.hstack((probs_1, probs_2, probs_3, probs_4))

In [0]:
probs_test = np.hstack((probs_1, probs_2, probs_3, probs_4))

In [0]:
# Обучение классификкатора

from sklearn.linear_model import LogisticRegression

clf = LogisticRegression().fit(probs_train, y_train)

In [0]:
predict_val = clf.predict_proba(probs_val)

In [0]:
predict_test = clf.predict_proba(probs_test) 

decoder = {classes_dict[cl]:cl for cl in classes_dict}
forecast = pd.Series(predict_test.argmax(axis=1)).map(decoder)

df_test['label'] = forecast
df_test.to_csv('Stacking_vSS41_SS42_EM41_EM42.csv', index=None)

In [0]:
# f1-score для разных наборов классификаторов

In [0]:
f1_score(predict_val.argmax(axis=1), y_val, average='macro') # классификаторы v 2 3 4 5

0.7533077700861295

In [0]:
f1_score(predict_val.argmax(axis=1), y_val, average='macro') # классификаторы v 1 4 SS41 EM42

0.7932197206315299

In [0]:
f1_score(predict_val.argmax(axis=1), y_val, average='macro') # классификаторы v 1 2 3 4 5 SS41 EM42

0.8168401720502562

In [0]:
f1_score(predict_val.argmax(axis=1), y_val, average='macro') # классификаторы v SS41 SS42 EM41 EM42

0.8141336888247914

In [0]:
f1_score(predict_val.argmax(axis=1), y_val, average='macro') # классификаторы v 1 2 3 4 5 SS41 SS42 EM41 EM42

0.8244484560798716

In [0]:
# Обучение классификаторов на разных частях датасета

In [0]:
X_train, X_val, y_train, y_val = train_test_split(df_train.fname.values, df_train.label_encoded.values, 
                                                  test_size=0.2, random_state=42)

X_train_12, X_train_34, y_train_12, y_train_34 = train_test_split(X_train, y_train, test_size=0.5, random_state=42)
X_train_1, X_train_2, y_train_1, y_train_2 = train_test_split(X_train_12, y_train_12, test_size=0.5, random_state=42)
X_train_3, X_train_4, y_train_3, y_train_4 = train_test_split(X_train_34, y_train_34, test_size=0.5, random_state=42)

X_train_clf1 = np.hstack((X_train_2, X_train_3, X_train_4))
X_train_clf2 = np.hstack((X_train_1, X_train_3, X_train_4))
X_train_clf3 = np.hstack((X_train_1, X_train_2, X_train_4))
X_train_clf4 = np.hstack((X_train_1, X_train_2, X_train_3))
y_train_clf1 = np.hstack((y_train_2, y_train_3, y_train_4))
y_train_clf2 = np.hstack((y_train_1, y_train_3, y_train_4))
y_train_clf3 = np.hstack((y_train_1, y_train_2, y_train_4))
y_train_clf4 = np.hstack((y_train_1, y_train_2, y_train_3))

train_loader = DataLoader(
                        EventDetectionDataset(os.path.join(DATA_PATH, train_data_folder), X_train, y_train),
                        batch_size=41
                )
train_loader_1 = DataLoader(
                        EventDetectionDataset(os.path.join(DATA_PATH, train_data_folder), X_train_clf1, y_train_clf1),
                        batch_size=41
                )
train_loader_2 = DataLoader(
                        EventDetectionDataset(os.path.join(DATA_PATH, train_data_folder), X_train_clf2, y_train_clf2),
                        batch_size=41
                )
train_loader_3 = DataLoader(
                        EventDetectionDataset(os.path.join(DATA_PATH, train_data_folder), X_train_clf3, y_train_clf3),
                        batch_size=41
                )
train_loader_4 = DataLoader(
                        EventDetectionDataset(os.path.join(DATA_PATH, train_data_folder), X_train_clf4, y_train_clf4),
                        batch_size=41
                )
val_loader_1 = DataLoader(
                        EventDetectionDataset(os.path.join(DATA_PATH, train_data_folder), X_train_1, y_train_1),
                        batch_size=41
                )
val_loader_2 = DataLoader(
                        EventDetectionDataset(os.path.join(DATA_PATH, train_data_folder), X_train_2, y_train_2),
                        batch_size=41
                )
val_loader_3 = DataLoader(
                        EventDetectionDataset(os.path.join(DATA_PATH, train_data_folder), X_train_3, y_train_3),
                        batch_size=41
                )
val_loader_4 = DataLoader(
                        EventDetectionDataset(os.path.join(DATA_PATH, train_data_folder), X_train_4, y_train_4),
                        batch_size=41
                )
val_loader = DataLoader(
                        EventDetectionDataset(os.path.join(DATA_PATH, train_data_folder), X_val, y_val),
                        batch_size=41
                )

In [0]:
criterion = nn.CrossEntropyLoss()
model = Model_1()
model = model.cuda()
model_name = 'baseline_v1_Stackikng_1.pt'
lr = 1e-3

optimizer = torch.optim.Adam(model.parameters(), lr=lr)

model_train(model, train_loader_1, val_loader_1, model_name, lr, 50)

In [0]:
criterion = nn.CrossEntropyLoss()
model = Model_3()
model = model.cuda()
model_name = 'baseline_v3_Stackikng_2.pt'
lr = 1e-3

optimizer = torch.optim.Adam(model.parameters(), lr=lr)

model_train(model, train_loader_2, val_loader_2, model_name, lr, 50)

In [0]:
criterion = nn.CrossEntropyLoss()
model = Model_4()
model = model.cuda()
model_name = 'baseline_v4_Stackikng_3.pt'
lr = 1e-3

optimizer = torch.optim.Adam(model.parameters(), lr=lr)

model_train(model, train_loader_3, val_loader_3, model_name, lr, 50)

In [0]:
criterion = nn.CrossEntropyLoss()
model = Model_5()
model = model.cuda()
model_name = 'baseline_v5_Stackikng_4.pt'
lr = 1e-3

optimizer = torch.optim.Adam(model.parameters(), lr=lr)

model_train(model, train_loader_4, val_loader_4, model_name, lr, 50)

In [0]:
# предсказание моделями train и val

In [0]:
model_name = 'baseline_v1_Stackikng_1.pt'
model = Model_1().cuda()
model.load_state_dict(torch.load(os.path.join('..', model_name)))
model.eval()
_, probs_1 = model_predict(model, val_loader)

model_name = 'baseline_v3_Stackikng_2.pt'
model = Model_3().cuda()
model.load_state_dict(torch.load(os.path.join('..', model_name)))
model.eval()
_, probs_2 = model_predict(model, val_loader)

model_name = 'baseline_v4_Stackikng_3.pt'
model = Model_4().cuda()
model.load_state_dict(torch.load(os.path.join('..', model_name)))
model.eval()
_, probs_3 = model_predict(model, val_loader)

model_name = 'baseline_v5_Stackikng_4.pt'
model = Model_5().cuda()
model.load_state_dict(torch.load(os.path.join('..', model_name)))
model.eval()
_, probs_4 = model_predict(model, val_loader)

In [0]:
probs_train = np.hstack((probs_1, probs_2, probs_3, probs_4))

In [0]:
probs_val = np.hstack((probs_1, probs_2, probs_3, probs_4))

In [0]:
# тренировка классификатора на train

In [0]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression().fit(probs_train, y_train)

In [0]:
# прогноз классификатора на val

In [0]:
predict_val = clf.predict_proba(probs_val)

f1_score(predict_val.argmax(axis=1), y_val, average='macro')

0.7401011371382926